In [9]:
import os
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException, TimeoutException
from selenium.webdriver.common.keys import Keys
import openai
import asyncio
import nest_asyncio

nest_asyncio.apply()

openai.api_key = ''
path = r"C:\Users\risha\seleniumtwitterstock\chromedriver.exe"

async def generate_email_content(professor_content):
    try:
        response = await openai.ChatCompletion.acreate(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"Generate an email from a student interested in research opportunities to a professor with the following details:\n\n{professor_content}\n\nDear Professor,"}
            ]
        )
        email_text = response.choices[0].message['content'].strip()
        return email_text
    except Exception as e:
        return f"An error occurred: {e}"

async def extract_professors_info(driver, university, field):
    search_query = f"{university} {field} faculty directory"
    driver.get('https://www.google.com')
    search_box = driver.find_element(By.NAME, 'q')
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(3)
    
    try:
        first_result = driver.find_element(By.XPATH, '(//a/h3)[1]/..')
        first_result.click()
    except NoSuchElementException:
        print(f"Faculty directory not found for {university} {field}")
        return
    
    time.sleep(5)
    
    successful_extractions = 0
    max_extractions = 10

    for i in range(1, 21):
        if successful_extractions >= max_extractions:
            print(f"Successfully extracted text for {max_extractions} professors from {university} {field}.")
            break
        
        try:
            if university.lower() == 'yale':
                professor_link_xpath = f'//*[@id="block-system-main"]/div/div/div/div[2]/table/tbody/tr[{i}]/td[2]/a[3]'
            elif university.lower() == 'princeton':
                professor_link_xpath = f'//*[@id="block-system-main"]/div/div[2]/div[1]/div[2]/div[3]/a[1]'
            elif university.lower() == 'ut austin':
                professor_link_xpath = f'//*[@id="block-utcs-bs-sass-content"]/div/article/div/div[2]/div/div/div/div/div/div/div[2]/div[1]/div[2]/span/a'
            elif university.lower() == 'mit':
                professor_link_xpath = f'/html/body/div[1]/div[2]/div[1]/div/main/section/div/div[2]/span/div/div[1]/h5/a'
            else:
                print(f"Unknown university structure for {university}")
                continue
            
            professor_link = driver.find_element(By.XPATH, professor_link_xpath)
            professor_link.click()
            
            time.sleep(3)
            professor_page_url = driver.current_url
            
            response = requests.get(professor_page_url)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                text = soup.get_text(separator=' ', strip=True)
                
                email = await generate_email_content(text)
                
                print(f"Generated email for professor {successful_extractions + 1} from {university} {field}:\n{email}\n{'-'*80}\n")
                
                successful_extractions += 1
            
            driver.back()
            time.sleep(3)
            
        except NoSuchElementException:
            print(f"Professor {i} link not found in {university} {field} directory.")
        except TimeoutException:
            print(f"Loading timeout for professor {i} in {university} {field} directory.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")

async def main():
    field = input("Enter the field of study: ")
    universities = input("Enter the universities (comma separated): ").split(',')

    service = Service(path)
    driver = webdriver.Chrome(service=service)
    driver.maximize_window()

    try:
        for university in universities:
            university = university.strip()
            print(f"Processing {university} for {field} field...")
            await extract_professors_info(driver, university, field)
    except WebDriverException as e:
        print(f"WebDriverException occurred: {e}")
    finally:
        driver.quit()

if __name__ == "__main__":
    if os.name == 'nt':  
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    asyncio.run(main())


Processing yale for cs field...
Generated email for professor 1 from yale cs:
An error occurred: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

--------------------------------------------------------------------------------

Generated email for professor 2 from yale cs:
An error occurred: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can 

: 